In [440]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc #맥 한글깨짐
rc('font', family='AppleGothic')#맥 한글깨짐

## 날씨 데이터
- 데이터 출처 : [기상청 기상자료개방포털](https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36)
- 제주도의 하루 비, 눈, 안개 데이터


In [441]:
df_rain = pd.read_csv("/Users/yeseulseo/Side_Project/DACON/Jeju_traffic/data/rain.csv" , encoding='cp949')
df_snow = pd.read_csv("/Users/yeseulseo/Side_Project/DACON/Jeju_traffic/data/snow.csv" ,encoding='cp949')
df_fog = pd.read_csv("/Users/yeseulseo/Side_Project/DACON/Jeju_traffic/data/fog.csv" ,encoding='cp949')

In [442]:
df_rain[['지점명','일시']].duplicated().sum()

0

In [443]:
df_rain['지점명'].value_counts()

지점명
고산     154
서귀포    152
제주     137
성산     125
Name: count, dtype: int64

In [ ]:
비 데이터

### 비 데이터
- 비 일기 예보는 고산, 성산도 따로 표시가 되어 있는데 고산은 제주시 성산은 서귀포시에 포함되므로 각각 시로 고친 후 중복 데이터는 삭제

In [444]:
df_rain=df_rain.replace({'지점명':{'고산':'제주','성산':'서귀포'}})
df_rain

,지점,지점명,일시,일강수량(mm)
0,184,제주,2021-09-01,14.5
1,184,제주,2021-09-02,37.8
2,184,제주,2021-09-03,21.8
3,184,제주,2021-09-05,7.6
4,184,제주,2021-09-06,5.3
...,...,...,...,...
563,189,서귀포,2022-08-19,0.0
564,189,서귀포,2022-08-24,7.2
565,189,서귀포,2022-08-26,0.0
566,189,서귀포,2022-08-30,0.4


In [445]:
df_rain1=df_rain[df_rain[['지점명','일시']].duplicated(keep=False)]
ind=df_rain1[(df_rain1['지점']==185)|(df_rain1['지점']==188)].index #중복 데이터중 고산, 성산의 데이터는 제주,서귀포에 포함이므로 삭제 
df_rain=df_rain.drop(ind)
df_rain.reset_index(drop=True,inplace=True)
df_rain

,지점,지점명,일시,일강수량(mm)
0,184,제주,2021-09-01,14.5
1,184,제주,2021-09-02,37.8
2,184,제주,2021-09-03,21.8
3,184,제주,2021-09-05,7.6
4,184,제주,2021-09-06,5.3
...,...,...,...,...
327,189,서귀포,2022-08-19,0.0
328,189,서귀포,2022-08-24,7.2
329,189,서귀포,2022-08-26,0.0
330,189,서귀포,2022-08-30,0.4


In [446]:
df_rain[['지점명','일시']].duplicated().sum()

0

In [447]:
df_rain['지점명'].value_counts()

지점명
제주     170
서귀포    162
Name: count, dtype: int64

In [448]:
df_rain.head()

,지점,지점명,일시,일강수량(mm)
0,184,제주,2021-09-01,14.5
1,184,제주,2021-09-02,37.8
2,184,제주,2021-09-03,21.8
3,184,제주,2021-09-05,7.6
4,184,제주,2021-09-06,5.3


In [449]:
df_snow.head()

,지점,지점명,일시,일 최심적설(cm)
0,184,제주,2021-12-26,4.4
1,184,제주,2021-12-27,1.2
2,184,제주,2022-01-11,0.4
3,184,제주,2022-02-20,0.3
4,189,서귀포,2021-12-17,0.4


In [450]:
df_fog.head()

,지점,지점명,일시,안개 계속시간(hr)
0,184,제주,2022-03-13,0.77
1,184,제주,2022-03-14,2.38
2,184,제주,2022-03-30,1.58
3,184,제주,2022-03-31,0.85
4,184,제주,2022-04-11,0.52


### 결측값 
- 없음


In [451]:
print(df_rain.isnull().sum())
print(df_snow.isnull().sum())
print(df_fog.isnull().sum())

지점          0
지점명         0
일시          0
일강수량(mm)    0
dtype: int64
지점            0
지점명           0
일시            0
일 최심적설(cm)    0
dtype: int64
지점             0
지점명            0
일시             0
안개 계속시간(hr)    0
dtype: int64


### column 
- 컬럼명 변경 
- 필요 없는 컬럼 제거

In [452]:
df_rain.drop(['지점'],axis=1,inplace=True)
df_snow.drop(['지점'],axis=1,inplace=True)
df_fog.drop(['지점'],axis=1,inplace=True)

In [453]:
df_rain.columns = ['address','base_date', 'weather']
df_snow.columns = ['address','base_date',  'weather']
df_fog.columns = ['address', 'base_date', 'weather']

### 데이터 정보

In [454]:
print(df_rain.info())
print(df_snow.info())
print(df_fog.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    332 non-null    object 
 1   base_date  332 non-null    object 
 2   weather    332 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    7 non-null      object 
 1   base_date  7 non-null      object 
 2   weather    7 non-null      float64
dtypes: float64(1), object(2)
memory usage: 296.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    31 non-null     object 
 1   base_date  31 non-null     object 
 2   weather   

In [455]:
df_rain_num=df_rain.copy()
df_rain_num

,address,base_date,weather
0,제주,2021-09-01,14.5
1,제주,2021-09-02,37.8
2,제주,2021-09-03,21.8
3,제주,2021-09-05,7.6
4,제주,2021-09-06,5.3
...,...,...,...
327,서귀포,2022-08-19,0.0
328,서귀포,2022-08-24,7.2
329,서귀포,2022-08-26,0.0
330,서귀포,2022-08-30,0.4


### 데이터 변경
- 눈, 비, 안개의 수치를 눈,비,안개로 데이터 변경 

In [456]:
def weather (df,k) :
    for i in range(len(df)) :

        if df['weather'][i] > 0 :
            df['weather'][i] = k
        else :
            df['weather'][i] = '없음'
    
    idx = df[df['weather'] == '없음'].index
    df= df.drop(idx)
   
    return df
    


In [457]:
df_rain = weather(df_rain,'비')
df_snow = weather(df_snow,'눈')
df_fog = weather(df_fog,'안개')

/var/folders/yp/r9q2z09j2rj4vfhr9q8r9vl40000gn/T/ipykernel_4234/2037857174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weather'][i] = k
/var/folders/yp/r9q2z09j2rj4vfhr9q8r9vl40000gn/T/ipykernel_4234/2037857174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weather'][i] = k
/var/folders/yp/r9q2z09j2rj4vfhr9q8r9vl40000gn/T/ipykernel_4234/2037857174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weather'][i]

In [458]:
print(df_rain['weather'].value_counts())
print(df_snow['weather'].value_counts())
print(df_fog['weather'].value_counts())

weather
비    224
Name: count, dtype: int64
weather
눈    7
Name: count, dtype: int64
weather
안개    31
Name: count, dtype: int64


### 데이터 합치기
- 안개와 눈은 겹치는 날이 없어서 그냥 병합
- 눈, 비 둘다 표기 된 날은 눈으로 안개와 비가 함께 표시된 날은 비로 수정하여 3개의 데이터 병합

In [459]:
snow_fog=pd.concat([df_snow,df_fog],axis=0)
snow_fog

,address,base_date,weather
0,제주,2021-12-26,눈
1,제주,2021-12-27,눈
2,제주,2022-01-11,눈
3,제주,2022-02-20,눈
4,서귀포,2021-12-17,눈
5,서귀포,2021-12-26,눈
6,서귀포,2021-12-27,눈
0,제주,2022-03-13,안개
1,제주,2022-03-14,안개
2,제주,2022-03-30,안개


In [460]:
merge_weather= pd.merge(df_rain,snow_fog, how='outer',on=['base_date','address'])
merge_weather

,address,base_date,weather_x,weather_y
0,제주,2021-09-01,비,NaN
1,제주,2021-09-02,비,NaN
2,제주,2021-09-03,비,NaN
3,제주,2021-09-05,비,NaN
4,제주,2021-09-06,비,NaN
...,...,...,...,...
233,서귀포,2022-03-31,NaN,안개
234,서귀포,2022-04-12,NaN,안개
235,서귀포,2022-05-30,NaN,안개
236,서귀포,2022-06-17,NaN,안개


In [461]:
merge_weather=merge_weather.fillna('없음')
merge_weather

,address,base_date,weather_x,weather_y
0,제주,2021-09-01,비,없음
1,제주,2021-09-02,비,없음
2,제주,2021-09-03,비,없음
3,제주,2021-09-05,비,없음
4,제주,2021-09-06,비,없음
...,...,...,...,...
233,서귀포,2022-03-31,없음,안개
234,서귀포,2022-04-12,없음,안개
235,서귀포,2022-05-30,없음,안개
236,서귀포,2022-06-17,없음,안개


In [462]:
for i in range(len(merge_weather)):
   if merge_weather['weather_x'][i]== '비':
      if merge_weather['weather_y'][i] == '눈':
         merge_weather['weather_x'][i] = '눈'

   else :
      merge_weather['weather_x'][i] = merge_weather['weather_y'][i]

merge_weather.drop(['weather_y'],axis=1,inplace=True) 
merge_weather.rename(columns={'weather_x':'weather'},inplace=True)

merge_weather

,address,base_date,weather
0,제주,2021-09-01,비
1,제주,2021-09-02,비
2,제주,2021-09-03,비
3,제주,2021-09-05,비
4,제주,2021-09-06,비
...,...,...,...
233,서귀포,2022-03-31,안개
234,서귀포,2022-04-12,안개
235,서귀포,2022-05-30,안개
236,서귀포,2022-06-17,안개


In [463]:
merge_weather['weather'].value_counts()

weather
비     218
안개     13
눈       7
Name: count, dtype: int64

In [464]:
merge_weather=merge_weather.sort_values('base_date')
merge_weather.reset_index(drop=True,inplace=True)
merge_weather

,address,base_date,weather
0,제주,2021-09-01,비
1,서귀포,2021-09-01,비
2,제주,2021-09-02,비
3,서귀포,2021-09-02,비
4,제주,2021-09-03,비
...,...,...,...
233,서귀포,2022-08-17,비
234,제주,2022-08-17,비
235,서귀포,2022-08-24,비
236,서귀포,2022-08-30,비


In [465]:
#csv파일로 저장 
merge_weather.to_csv("weather.csv", index = False)

In [466]:
#비 수치 
df_rain_num.to_csv('rain.csv', index=False)